In [1]:
from toolbox import ToolBox
from tqdm import tqdm

from models import *

tqdm.pandas()

In [2]:
import pandas as pd

In [3]:
tb = ToolBox()

In [28]:
# Get all genres for all games
df = tb.load_data_sql(table='game_genres', use_cache=True)

Loaded from cache


In [29]:
# Get the genre frequency
genre_freq = df['genre'].value_counts().to_frame()

# Rest index
genre_freq.reset_index(inplace = True)

# Rename columns
genre_freq.rename(columns={'index' : 'genre', 'genre' : 'freq'}, inplace = True)

In [31]:
genre_freq.shape, 'genres', df.shape, 'dataframe with multiple genres per game'

((169, 2), 'genres', (75484, 2), 'dataframe with multiple genres per game')

In [32]:
# Merge initial dataframe with the frequency counts for the genres
df = pd.merge(df, genre_freq).sort_values('game')

In [33]:
df.shape

(75484, 3)

In [7]:
# Find indexes for each game-genre combination with the highest frequency
idx = df.groupby(['game'])['freq'].transform(max) == df['freq']

In [8]:
# Create new dataframe with the assigned genre for each game
final_df = df[idx].copy()

In [9]:
# Drop the frequency column
final_df.drop('freq', axis = 1, inplace = True)

In [11]:
final_df.shape

(22378, 2)

##### Get console info

In [16]:
# Get all genres for all games
console_data = tb.load_data_sql(table='games', select='game, console', use_cache=True)

Loading from database


In [18]:
console_data.head()

,game,console
0,-arcane-preraise--pc,pc
1,-arcane-raise--pc,pc
2,-arcane-reraise--pc,pc
3,0-ji-no-kane-to-cinderella-halloween-wedding-psp,psp
4,0-kara-no-shogi-shogi-youchien-ayumi-kumi-r-pl...,playstation


#### Load dataset

In [117]:
cv_data = pd.read_csv('cross_sectional_data.csv')

In [119]:
# look at initial shape of dataframe
cv_data.shape

((22378, 2), 'genres', (39104, 2), 'consoles', (13068, 8), 'regression data')

In [121]:
# add genre column to the dataframe used to run models
cv_data = cv_data.merge(final_df, on='game', how='left')

In [133]:
# add console column to the dataframe used to run models
cv_data = cv_data.merge(console_data, on='game', how='left')

In [135]:
# confirm shape is same in lenght and increased by 2 due to added columns (genre, console)
cv_data.shape

(13068, 10)

In [139]:
# write dataframe to csv
cv_data.to_csv('cross-sectional-data-with-control-variables.csv')